## Fidenza trades - data collection

In [1]:
import pandas as pd

In [2]:
import os
path = str(os.getcwd()) + '/'

### 1. Fidenza trades in ETH

Source: Ethereum public dataset built by Google BigQuery https://evgemedvedev.medium.com/ethereum-blockchain-on-google-bigquery-283fb300f579

For those with paid access to Google BigQuery, you can connect & query directly from this Python notebook using the code below:

In [3]:
# from google.cloud import bigquery

# # set your service account key
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'file_path/my_gcp_key.json'

In [4]:
# %load_ext google.cloud.bigquery

In [5]:
# %%bigquery fdz_trades
# SELECT
#   a.block_timestamp as block_time,
#   a.value AS token_id,
#   b.value / POW(10,18) AS original_amount
# FROM
#   `bigquery-public-data.crypto_ethereum.token_transfers` a
# LEFT JOIN
#   `bigquery-public-data.crypto_ethereum.transactions` b
# ON
#   a.transaction_hash = b.HASH
# WHERE
#   token_address = '0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270'
#   AND (SAFE_CAST(a.value AS NUMERIC) BETWEEN 78000000 AND 78000998)
#   AND (DATE(a.block_timestamp) BETWEEN "2021-06-11" AND "2021-07-10")
# ORDER BY
#   a.block_timestamp DESC

For those with the free access like myself, let's work around the limited quota by executing the SQL query on BigQuery console, then downloading the data to your local folder before processing:

In [6]:
fdz_trades = pd.read_csv(path + 'Fidenza_updated/fdz_trades_to_20220330.csv')

In [7]:
fdz_trades.head(3)

,block_time,token_id,original_amount
0,2022-03-28 21:09:00,78000336,55.000
1,2022-03-25 09:52:00,78000893,249.005
2,2022-03-25 00:13:00,78000378,53.000


In [8]:
fdz_trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   block_time       2022 non-null   object 
 1   token_id         2022 non-null   int64  
 2   original_amount  2022 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 47.5+ KB


***Define a function to clean current data, as well as update data later on***

In [9]:
def trades_prep(df):
    # drop rows with zero amount as they are likely to be transfer or in bundle transactions
    df = df.loc[df['original_amount'] > 0.01]
    
    # reformat datetime column
    df.loc[:,'block_time'] = df['block_time'].astype(str).str[:-1].str.replace('T', ' ')
    df.loc[:,'block_time'] = pd.to_datetime(df['block_time'], format='%Y-%m-%d %H:%M:%S')

    # get the minute to join with ETHUSD prices later
    df.loc[:,'block_time'] = df['block_time'].dt.floor('T')
    
    return df

***Clean the current dataset - currently updated until March 30, 2022 17:00***

In [10]:
fdz_trades = trades_prep(fdz_trades)

In [11]:
fdz_trades.head(3)

,block_time,token_id,original_amount
0,2022-03-28 21:09:00,78000336,55.000
1,2022-03-25 09:52:00,78000893,249.005
2,2022-03-25 00:13:00,78000378,53.000


***Update data***

In [12]:
# trades_update = pd.read_csv(path + '/Fidenza_updated/Fidenza_trades_to_date.csv')

In [13]:
# # clean the newly added data
# clean_update = trades_prep(trades_update)

# # concat with the current table - specify the new date, eg. currently updated to 2022-03-28
# fdz_trades_to_new_date = pd.concat([clean_update, fdz_trades])

# # reset index
# fdz_trades_to_new_date.reset_index(drop=True, inplace=True)

Update local file

In [14]:
# fdz_trades_to_new_date.to_csv(path + 'Fidenza_updated/fdz_trades_to_new_date.csv', index=False)